In [1]:
#import sklearn
#from sklearn import datasets
import tensorflow as tf
import numpy as np

/Users/yuezha01/anaconda3/envs/tensorflow_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/yuezha01/anaconda3/envs/tensorflow_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
w_true = np.array([1.0, 1.0]).reshape((2, 1))
b_true = 1.0

In [4]:
data_size = 1000
np.random.seed(10)
train_data_x = np.random.random(size=(data_size, 2))

In [5]:
train_data_x.shape

(1000, 2)

In [6]:
train_data_y = np.matmul(train_data_x, w_true) + b_true
train_data_y.shape

(1000, 1)

In [7]:
np.random.seed(101)
test_data_x = np.random.random(size=(data_size, 2))
test_data_y = np.matmul(test_data_x, w_true) + b_true

In [8]:
print(train_data_x.shape, train_data_y.shape)

(1000, 2) (1000, 1)


In [9]:
print(test_data_x.shape, test_data_y.shape)

(1000, 2) (1000, 1)


#### Neural network

In [10]:
n_features = 2
n_class = 1

In [11]:
tf.reset_default_graph()

In [12]:
x = tf.placeholder(dtype=tf.float32, shape=[None, n_features], name='x')
y = tf.placeholder(dtype=tf.float32, shape=[None, n_class], name='y')

In [13]:
w_fc1 = tf.Variable(tf.truncated_normal(shape=[n_features, n_class], stddev=0.1), name='w_fc1')
b_fc1 = tf.Variable(tf.constant(0.1, shape=[n_class]), name='b_fc1')

In [14]:
y_pred = tf.add(tf.matmul(x, w_fc1), b_fc1, name='y_pred')

In [15]:
loss = tf.reduce_mean(tf.square(y - y_pred))
#train_step = tf.train.AdamOptimizer(1e-4).minimize(loss, name="train_step")
train_step = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss, name='train_step')

In [16]:
n_epoch = 10
batch_size = 2
n_batch = int(data_size / batch_size)

In [17]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('Initial loss = {}'.format(sess.run(loss, feed_dict={x: test_data_x, y:test_data_y})))
saver = tf.train.Saver()

Initial loss = 3.289672374725342


In [18]:
for e in range(n_epoch):
    batch_start, batch_end = 0, batch_size
    for b in range(n_batch):
        sess.run(train_step, feed_dict = {x: train_data_x[batch_start:batch_end, :],
                                          y: train_data_y[batch_start:batch_end, :]})
        batch_start += batch_size
        batch_end += batch_size
    l = sess.run(loss, feed_dict = {x: test_data_x, y: test_data_y})
    print('epoch{0}: loss={1}, w={2}, b={3}'.format(e, l, sess.run(w_fc1), sess.run(b_fc1)))

epoch0: loss=0.003971228376030922, w=[[0.81850445]
 [0.87285185]], b=[1.163132]
epoch1: loss=0.0012793572386726737, w=[[0.897092]
 [0.927687]], b=[1.092613]
epoch2: loss=0.00041217400575987995, w=[[0.9416348 ]
 [0.95889324]], b=[1.0525765]
epoch3: loss=0.00013279521954245865, w=[[0.966891 ]
 [0.9766405]], b=[1.0298468]
epoch4: loss=4.27838895120658e-05, w=[[0.9812152]
 [0.9867299]], b=[1.0169429]
epoch5: loss=1.3785638657282107e-05, w=[[0.98934066]
 [0.9924626 ]], b=[1.0096183]
epoch6: loss=4.442105819180142e-06, w=[[0.9939506 ]
 [0.99571943]], b=[1.00546]
epoch7: loss=1.4310061260403018e-06, w=[[0.9965669 ]
 [0.99756974]], b=[1.003099]
epoch8: loss=4.608922381521552e-07, w=[[0.9980522]
 [0.9986201]], b=[1.0017588]
epoch9: loss=1.485476133211705e-07, w=[[0.99889445]
 [0.99921614]], b=[1.0009985]


In [19]:
sess.run(y_pred, feed_dict={x: np.array([[1.0, 2.0]])})

array([[3.9983253]], dtype=float32)

In [20]:
saver.save(sess, "./lr_model/chkps")
out_nodes = [y_pred.op.name]
print(out_nodes)

['y_pred']


In [21]:
from tensorflow.python.framework import graph_util as gu
from tensorflow.python.framework.graph_util import remove_training_nodes
from tensorflow.tools.graph_transforms import TransformGraph

In [22]:
sub_graph_def = remove_training_nodes(sess.graph_def)

In [23]:
sub_graph_def = gu.convert_variables_to_constants(sess, sub_graph_def, out_nodes)

INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.


In [25]:
graph_path = tf.train.write_graph(sub_graph_def,
                                  "./lr_model",
                                  "lr_model.pb",
                                  as_text=False)

print('written graph to: %s' % graph_path)

written graph to: ./lr_model/lr_model.pb


In [ ]:
sess.close()

#### Use uTensor to generate C++ source code from the saved model
* conda activate tensorflow_cpu
* utensor-cli convert <path to .pb file> --output-nodes=y_pred, for example, utensor-cli convert ./tensorflow-models/lr_model/lr_model.pb --output-nodes=y_pred

#### mbed compile the model with mbed-os
* modify the main.cpp with #include and get_<model-name>_ctx()
* mbed compile -m DISCO_F413ZH -t GCC_ARM --profile=uTensor/build_profile/release.json

#### Question
* lr_model_weight.hpp shows the right value
* why the value shown in Serial terminal is 1090520913, which is supposed to be 8?
* use float in .hpp and int in main.cpp - 1090520913
  use int in .hpp and int in main.cpp - 1547681806
  use int in .hpp and float in main.cpp - 1075840573